In [1]:
%%time
import pandas as pd
import numpy as np
from dask import dataframe as dd

LargeDataFile = 'D:\\Downloads\\pain-pills-in-the-usa\\arcos_all_washpost.tsv'
#print(LargeDataFile)


interestingCols = ['BUYER_CITY', 'BUYER_STATE',
       'BUYER_ZIP', 'BUYER_COUNTY','DRUG_NAME', 'QUANTITY','TRANSACTION_DATE', 'dos_str', 'Revised_Company_Name']

traintypes = {'DRUG_NAME': 'str',
              'dos_str': 'float64', 
              'QUANTITY': 'int64',
              'BUYER_CITY': 'str',
              'BUYER_STATE': 'str',
              'BUYER_COUNTY': 'str',
              'BUYER_ZIP': 'int64',
              'TRANSACTION_DATE': 'int64',
              'Revised_Company_Name': 'str'}


dfd = dd.read_csv(LargeDataFile,delimiter='\t',usecols=interestingCols,dtype=traintypes)
dfd.head()

Wall time: 19 s


,BUYER_CITY,BUYER_STATE,BUYER_ZIP,BUYER_COUNTY,DRUG_NAME,QUANTITY,TRANSACTION_DATE,Revised_Company_Name,dos_str
0,MALDEN,MA,2148,MIDDLESEX,HYDROCODONE,1,12262012,Mallinckrodt,10.0
1,PHOENIX,AZ,85085,MARICOPA,HYDROCODONE,4,3112009,Apotheca Inc.,5.0
2,GILBERT,AZ,85233,MARICOPA,HYDROCODONE,40,11252008,Apotheca Inc.,5.0
3,GILBERT,AZ,85233,MARICOPA,HYDROCODONE,20,6122009,Apotheca Inc.,7.5
4,GILBERT,AZ,85233,MARICOPA,HYDROCODONE,10,10022009,Apotheca Inc.,5.0


In [2]:
%%time
# Converting TRANSACTION_DATE to datetime format and extracting year and storing it in a new column
dfd['TRANSACTION_DATE']=dd.to_datetime(dfd['TRANSACTION_DATE'].astype('str'),format='%m%d%Y')
dfd['YEAR']=dfd['TRANSACTION_DATE'].dt.year
dfd.head()

Wall time: 4.65 s


,BUYER_CITY,BUYER_STATE,BUYER_ZIP,BUYER_COUNTY,DRUG_NAME,QUANTITY,TRANSACTION_DATE,Revised_Company_Name,dos_str,YEAR
0,MALDEN,MA,2148,MIDDLESEX,HYDROCODONE,1,2012-12-26,Mallinckrodt,10.0,2012
1,PHOENIX,AZ,85085,MARICOPA,HYDROCODONE,4,2009-03-11,Apotheca Inc.,5.0,2009
2,GILBERT,AZ,85233,MARICOPA,HYDROCODONE,40,2008-11-25,Apotheca Inc.,5.0,2008
3,GILBERT,AZ,85233,MARICOPA,HYDROCODONE,20,2009-06-12,Apotheca Inc.,7.5,2009
4,GILBERT,AZ,85233,MARICOPA,HYDROCODONE,10,2009-10-02,Apotheca Inc.,5.0,2009


In [ ]:
%%time
# Convert to pandas dataframe
pdf = dfd[dfd.YEAR == 2010].compute()

In [ ]:
%%time
# Resetting the index and exporting it pandas dataframe into a tsv file
pdf=pdf.reset_index(drop=True)
pdf.to_csv('D:\\Downloads\\pain-pills-in-the-usa\\short.tsv',index=False,sep='\t')


In [2]:
%%time
pdf=pd.read_csv('D:\\Downloads\\pain-pills-in-the-usa\\short.tsv',sep='\t')
pdf.head()

Wall time: 2min 5s


,BUYER_CITY,BUYER_STATE,BUYER_ZIP,BUYER_COUNTY,DRUG_NAME,QUANTITY,TRANSACTION_DATE,Revised_Company_Name,dos_str,YEAR
0,GILBERT,AZ,85233,MARICOPA,HYDROCODONE,20,2010-03-23,Apotheca Inc.,5.0,2010
1,PHOENIX,AZ,85085,MARICOPA,HYDROCODONE,3,2010-09-15,Apotheca Inc.,5.0,2010
2,PHOENIX,AZ,85085,MARICOPA,HYDROCODONE,3,2010-12-09,Apotheca Inc.,7.5,2010
3,PULASKI,NY,13142,OSWEGO,HYDROCODONE,1,2010-12-02,"Allergan, Inc.",10.0,2010
4,PULASKI,NY,13142,OSWEGO,HYDROCODONE,5,2010-11-05,"Allergan, Inc.",10.0,2010


In [17]:
# Number of occurences of each zip
%%time
grouped=pdf.groupby(['BUYER_ZIP','BUYER_COUNTY','BUYER_CITY','BUYER_STATE'])['BUYER_ZIP'].count().reset_index(name='COUNT')
#sort in descending order
grouped=grouped.sort_values('COUNT', ascending=False).reset_index(drop=True)
grouped.head()

Wall time: 32.2 s


,BUYER_ZIP,BUYER_COUNTY,BUYER_CITY,BUYER_STATE,COUNT
0,37660,SULLIVAN,KINGSPORT,TN,16328
1,21061,ANNE ARUNDEL,GLEN BURNIE,MD,15239
2,48180,WAYNE,TAYLOR,MI,13036
3,72401,CRAIGHEAD,JONESBORO,AR,12982
4,71913,GARLAND,HOT SPRINGS,AR,12566


In [41]:
# Number of occurences of each county
grouped.groupby(['BUYER_COUNTY','BUYER_CITY','BUYER_STATE'])['COUNT'].sum().reset_index().sort_values('COUNT', ascending=False).reset_index(drop=True)

,BUYER_COUNTY,BUYER_CITY,BUYER_STATE,COUNT
0,HARRIS,HOUSTON,TX,180350
1,CLARK,LAS VEGAS,NV,123598
2,PHILADELPHIA,PHILADELPHIA,PA,120448
3,MARICOPA,PHOENIX,AZ,108097
4,MARION,INDIANAPOLIS,IN,102433
...,...,...,...,...
13039,DELTA,GLADSTONE,MI,1
13040,COOK,GLENWOOD,IL,1
13041,DANE,BLACK EARTH,WI,1
13042,RUTLAND,WEST PAWLET,VT,1


In [42]:
# Importing census data 2010
df_merge=pd.read_csv('D:\Downloads\pain-pills-in-the-usa\\zip_data.csv')
print(df_merge.head())


    ZIP  MEDIAN_HOUSEHOLD_INCOME  POPULATION  POPULATION_DENSITY_PER_SQ_MILE
0  1001                    56663       16769                     1465.565461
1  1002                    49853       29049                      527.751031
2  1003                    28462       10372                    14587.904360
3  1005                    75423        5079                      114.800416
4  1007                    79076       14649                      278.270615


In [43]:
%%time
# Dropping non-relevant columns
df=pdf.drop(columns=['YEAR','Revised_Company_Name','BUYER_CITY','BUYER_COUNTY','BUYER_STATE','TRANSACTION_DATE'])
df.head()

Wall time: 3.01 s


,BUYER_ZIP,DRUG_NAME,QUANTITY,dos_str
0,85233,HYDROCODONE,20,5.0
1,85085,HYDROCODONE,3,5.0
2,85085,HYDROCODONE,3,7.5
3,13142,HYDROCODONE,1,10.0
4,13142,HYDROCODONE,5,10.0


In [44]:
# Creating a new column which is product of Quantity and dos_str
df['MG']=df['QUANTITY']*df['dos_str']
df.head()

,BUYER_ZIP,DRUG_NAME,QUANTITY,dos_str,MG
0,85233,HYDROCODONE,20,5.0,100.0
1,85085,HYDROCODONE,3,5.0,15.0
2,85085,HYDROCODONE,3,7.5,22.5
3,13142,HYDROCODONE,1,10.0,10.0
4,13142,HYDROCODONE,5,10.0,50.0


In [45]:
%%time
# Dropping columns and groupby MG column by Buyer zip and Drug name
df=df.drop(columns=['QUANTITY','dos_str'])
df2=df.groupby(['BUYER_ZIP','DRUG_NAME'])['MG'].sum().reset_index(name='sum_MG')

Wall time: 17.4 s


In [46]:
df2.head()

,BUYER_ZIP,DRUG_NAME,sum_MG
0,601,HYDROCODONE,1537.5
1,601,OXYCODONE,1530.0
2,602,HYDROCODONE,587.5
3,602,OXYCODONE,1120.0
4,603,HYDROCODONE,710.0


In [47]:
#pivotting
df3=df2.pivot(index='BUYER_ZIP',columns='DRUG_NAME', values='sum_MG')

In [48]:
df3.head()

DRUG_NAME,HYDROCODONE,OXYCODONE
BUYER_ZIP,,
601,1537.5,1530.0
602,587.5,1120.0
603,710.0,5460.0
604,45.0,100.0
605,722.5,3492.5


In [51]:
# Creating a new column from existing indexes
df3['ZIP']=df3.index

In [52]:
df3.head()

DRUG_NAME,HYDROCODONE,OXYCODONE,ZIP
BUYER_ZIP,,,
601,1537.5,1530.0,601
602,587.5,1120.0,602
603,710.0,5460.0,603
604,45.0,100.0,604
605,722.5,3492.5,605


In [53]:
# merging census data with pivot dataframe 
result=pd.merge(df_merge,df3,on='ZIP',how='left')

In [54]:
# Replacing null values with 0
result=result.fillna(0)
result.head(10)

,ZIP,MEDIAN_HOUSEHOLD_INCOME,POPULATION,POPULATION_DENSITY_PER_SQ_MILE,HYDROCODONE,OXYCODONE
0,1001,56663,16769,1465.565461,5737.5,64422.5
1,1002,49853,29049,527.751031,2517.5,28040.0
2,1003,28462,10372,14587.904360,0.0,0.0
3,1005,75423,5079,114.800416,4307.5,45322.5
4,1007,79076,14649,278.270615,4782.5,47247.5
5,1008,63980,1263,23.477582,5.0,0.0
6,1009,51452,741,929.736512,0.0,0.0
7,1010,75625,3609,103.877040,0.0,0.0
8,1011,63476,1370,43.332490,0.0,0.0
9,1012,58750,661,50.384938,0.0,0.0


In [55]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32633 entries, 0 to 32632
Data columns (total 6 columns):
ZIP                               32633 non-null int64
MEDIAN_HOUSEHOLD_INCOME           32633 non-null int64
POPULATION                        32633 non-null int64
POPULATION_DENSITY_PER_SQ_MILE    32633 non-null float64
HYDROCODONE                       32633 non-null float64
OXYCODONE                         32633 non-null float64
dtypes: float64(3), int64(3)
memory usage: 1.7 MB


In [56]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32633 entries, 0 to 32632
Data columns (total 4 columns):
ZIP                               32633 non-null int64
MEDIAN_HOUSEHOLD_INCOME           32633 non-null int64
POPULATION                        32633 non-null int64
POPULATION_DENSITY_PER_SQ_MILE    32633 non-null float64
dtypes: float64(1), int64(3)
memory usage: 1019.9 KB


In [58]:
# Exporitng the new dataframe and grouped data as a csv file
result.to_csv('D:\\Downloads\\pain-pills-in-the-usa\\combine_zip.csv',index=False)
grouped.to_csv('D:\\Downloads\\pain-pills-in-the-usa\\grouped.csv',index=False)